In [ ]:
#Mount the google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
drive='/content/gdrive/My Drive/python/fever/'  #Change the path to the correct directory where the cleaned_data.csv file is present
os.chdir(drive)
print(os.getcwd())

/content/gdrive/My Drive/python/fever


In [ ]:
!pip install -U sentence-transformers

     |████████████████████████████████| 71kB 7.1MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.9-cp36-none-any.whl size=101036 sha256=2313c71d511a57dc5776f5eae75246300663632a85b9e33f8a0c700ad8ada7f7
  Stored in directory: /root/.cache/pip/wheels/fc/89/43/f2f5bc00b03ef9724b0f6254a97eaf159a4c4ddc024b33e07a
Successfully built sentence-transformers


In [ ]:
!pip install transformers

     |████████████████████████████████| 1.3MB 13.9MB/s 
     |████████████████████████████████| 890kB 47.2MB/s 
     |████████████████████████████████| 1.1MB 50.8MB/s 
     |████████████████████████████████| 2.9MB 61.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=e8b67f7ef6bf25db59dc622b2c903e5e838fdfadfe5107f8e67c20c46bc0c1ab
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!ls

body_index			    neural_network.py
cache_dir			    __pycache__
combined_replaced_pronouns.jsonl    runs
fever_bert_base_encased		    tfidf_vectorizer_5000_features.pickle
formatted_data_train_3_class.jsonl  title_index
license.html			    wiki-pages
__MACOSX			    wiki-pages.zip


In [ ]:
from sentence_transformers import SentenceTransformer
def get_vectors(sentences):
  model_name = 'bert-base-nli-cls-token'
  model = SentenceTransformer(model_name)
  embeddings = model.encode(sentences)
  return embeddings 


In [ ]:
print(get_vectors(["This is a cat", "This is a dog"]))

[[ 0.36742917 -0.1829594   0.8470633  ...  0.5516481   0.4876297
   0.2861368 ]
 [ 0.39284614 -0.12082902  0.43790206 ... -0.71040785  0.16923594
   0.6204951 ]]


In [ ]:
import json
import sys
import pickle
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import neural_network as nn
def pre_process(sentence):
    ##Replace brackets
    brackets = ['-LRB-', '-LSB-', '-RRB-', '-RSB-']
    for bracket in brackets:
        sentence = sentence.replace(bracket, " ")
    return sentence

def get_training_data():
    samples_to_use = 250000  #max=368892
    label_dict = {"SUPPORTS":0, "REFUTES": 1, "NOT ENOUGH INFO": 2}

    X_all = []
    y_all = []
    count = 0
    #train_file = "formatted_data_train.jsonl"
    train_file = "formatted_data_train_3_class.jsonl"
    fp = open(train_file, 'r')
    for line in fp:
        if count >= samples_to_use:
            break
        obj = json.loads(line.strip())
        claim = pre_process(obj['claim'])
        evidence = pre_process(obj['evidence'])
        X = (claim, evidence)
        y = label_dict[obj['label']]
        X_all.append(X)
        y_all.append(y)
        count += 1
    return X_all, y_all


In [ ]:

def train_infer_bert():
    (X_all, y_all) = get_training_data()

    claims = [claim for (claim, _) in X_all]
    evidences = [evidence for (_, evidence) in X_all]
    print("Transforming claims")
    u = get_vectors(claims)
    print("Transforming evidences")
    v = get_vectors(evidences)
    print("Shape of u={} Shape of v={}".format(u.shape, v.shape))
    uplusv = u + v
    uminusv = u - v
    ubyv = u * v
    print("Resulting shapes=", uplusv.shape, uminusv.shape, ubyv.shape)
    all_features = np.concatenate((u, v, uplusv, uminusv, ubyv), axis=1)
    print("Shape of all=", all_features.shape)
    X_train, X_test, y_train, y_test = train_test_split(all_features, y_all, test_size=0.2, random_state=4)
    print("#Train=", len(X_train), len(y_train))
    print("#Test=", len(X_test), len(y_test))
    classifier_model_name = "baseline_classifier_inferbert_model.h5"
    nn.EPOCHS=20
    nn.fit_predict(X_train, y_train, X_test, y_test, classifier_model_name)

train_infer_bert()

Transforming claims
Transforming evidences
Shape of u=(250000, 768) Shape of v=(250000, 768)
Resulting shapes= (250000, 768) (250000, 768) (250000, 768)
Shape of all= (250000, 3840)
#Train= 200000 200000
#Test= 50000 50000
Epoch 1/20
196/196 [==============================] - 2s 9ms/step - loss: 0.4847 - accuracy: 0.8838
Epoch 2/20
196/196 [==============================] - 2s 9ms/step - loss: 0.3283 - accuracy: 0.9089
Epoch 3/20
196/196 [==============================] - 2s 9ms/step - loss: 0.2895 - accuracy: 0.9205
Epoch 4/20
196/196 [==============================] - 2s 9ms/step - loss: 0.2623 - accuracy: 0.9283
Epoch 5/20
196/196 [==============================] - 2s 9ms/step - loss: 0.2386 - accuracy: 0.9356
Epoch 6/20
196/196 [==============================] - 2s 9ms/step - loss: 0.2212 - accuracy: 0.9410
Epoch 7/20
196/196 [==============================] - 2s 9ms/step - loss: 0.2047 - accuracy: 0.9462
Epoch 8/20
196/196 [==============================] - 2s 9ms/step - loss: 0.1